# SAR tomography of tropical forests

Author: Xiao Liu\
E-mail: xiao.liu@mailbox.tu-dresden.de

This script is developed based on the [TomoSAR tuorial EO-college](https://github.com/EO-College/tomography_tutorial). 

Data: F-SAR P-band TomoSAR data from AfriSAR 2016 campaign.\
Study site: Lopé national park, Gabon.

## Step 1: import general python modules

In [ ]:
# install required python packages
#!pip install -r E:/TomoSAR/code/requirements.txt

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sys import exit

import warnings
warnings.filterwarnings("ignore")

## Step 2: define working directory

In [ ]:
# set the project path
project_path = 'E:/TomoSAR/'

inpath = project_path + 'data'

workspace = project_path + 'code'
os.chdir(workspace)

## Step 3: import functions for tomography and plotting

In [ ]:
from tomosar_toolbox import tomobox, normalize, topo_residual_correction,\
                         covmat_downsampling
from tomosar_plotting import cov_mat_plot, tomo_plot,\
                             grouped_tomosar_profiles, tomosar_phase_centre, \
                             tomosar_layerd_reflectivity, quick_look, insar_quick_look

## Step 4: define input parameters

In [ ]:
ps_rg = 1.19 #Pixel spacing in range
ps_az = 0.9 #pixel spacing in azimuth

# define the boxcar smoothing dimension (in meters)
multi_look = 25 #25, 50

# define the max height for the inversion
height = 65
z_vector = np.arange(-height, height + 1, 1)

# select polarization
pol = 'hh'
# pol = 'hv'
# pol = 'vv'

# select tomosar algorithm
tomo_method = 'capon'#'capon','beamforming',

# flag of terrain normalisation
terrain_cor_flag = 0 # 0: without terrain normalisation, 1: with terrain normalisation
# terrain_cor_flag = 1 

if terrain_cor_flag == 0: 
    outpath = project_path + 'out' # save results without terrain normalisation
else:
    outpath = project_path + 'out/terrain_normalised' # save results with terrain normalisation
    

## Step 5: read sar data

In [ ]:
outname = os.path.join(inpath, '{}_sample_data.npz'.format(pol))
data = np.load(outname)

slc_stack = data['slc_stack'] # single complex look (SLC) image
kz_stack = data['kz_stack'] # vertical wavenumber
phase_stack = data['phase_stack'] # topographical phase

# subset
#slc_stack = slc_stack[:,:,:5]
#kz_stack = kz_stack[:,:,:5]
#phase_stack = phase_stack[:,:,:5]

del data

n_row, n_col, n_track = slc_stack.shape

print(slc_stack.shape)

## Step 6: look at the data: slc intensity, phase, kz, topographical phase 

In [ ]:
slc_id = 3
img_path = outpath + '/Track_{}_quick_look.png'.format(slc_id)         
quick_look (slc_id, slc_stack, kz_stack, phase_stack, img_path)

## Step 7: remove flat-earth and topography phase

In [ ]:
print('Remove flat-earth and topography phase')
outname = os.path.join(outpath, '{}_normalized_stack.npy'.format(pol))

normalized_stack = slc_stack.copy()
# start from second slc (kz and dem_phase of master slc are 0)
for n in tqdm(range(1,n_track)):
    dem_phase = phase_stack[:,:,n].squeeze()
    slc_uncal = slc_stack[:,:,n].squeeze()
    normalized_stack[:,:,n] = slc_uncal*np.exp(1j*dem_phase)#phase calibration
del dem_phase,slc_uncal

np.save(outname, normalized_stack)

## Step 8: remove residual topography phase

In [ ]:
if terrain_cor_flag == 1 :
    hh_tomo_path = os.path.join(project_path + 'out/',
                                'hh_tomo_ml{}_h{}_capon.npy'.format(multi_look, height))
    
    if os.path.isfile(hh_tomo_path) == False:
        print('Please first calculate the hh tomogrpahy without terrain correction.')
        terrain_cor_flag = 0
        exit(0)

    else:
        print('Remove residual topography phase')
        terrain_path = os.path.join(project_path + 'out/',
                                    'tomo_ml{}_h{}'.format(multi_look, height)+'_hh_capon_terrain.npy')
        
        normalized_stack, terrain = topo_residual_correction(normalized_stack, kz_stack, z_vector, 
                                                             hh_tomo_path, terrain_path)
        
        #% plot
        plt.figure(dpi = 300)
        plt.imshow(terrain, cmap='jet', vmax = 30, vmin = -30)
        plt.colorbar()
        plt.title('Topography residual (m)')
        #plt.tight_layout() 

## Step 9: look at the InSAR phase before and after removing flat-earth and topography phase

In [ ]:
slc_1_id, slc_2_id = 1, 3
insar_quick_look(slc_1_id, slc_2_id, slc_stack, normalized_stack)

## Step 10: compute covariance matrix 

In [ ]:
print('Calculate covariance matrix')
covariance_matrix, r_out_smpl, x_out_smpl = covmat_downsampling(normalized_stack, multi_look,  ps_rg, ps_az)
outname = os.path.join(outpath, '{}_cov_matrix_ml{}.npy'.format(pol, multi_look))
np.save(outname, covariance_matrix)

# update kz using downsampling index along azimuth and range
kz_stack_down = kz_stack[r_out_smpl,:,:][:,x_out_smpl,:]

## Step 11: plot covariance matrix

In [ ]:
img_path = outpath + '/{}_covariance_matrix_quick_look.png'.format(pol)
cov_mat_plot(covariance_matrix, img_path)

## Step 12: calculate TomoSAR reflectivity profiles

In [ ]:
tomo = tomobox(covariance_matrix,kz_stack_down,z_vector,outname,tomo_method)

outname = os.path.join(outpath, '{}_tomo_ml{}_h{}_{}.npy'.format(pol, multi_look, height, tomo_method))
np.save(outname, tomo)

## Step 13: normalise the tomography reflectivity to 0 and 1 per-pixel

In [ ]:
tomo_norm = np.apply_along_axis(normalize, 2, tomo)

## Step 14: plotting of the tomography result

In [ ]:
rg_ratio, az_ratio = 0.5, 0.6

#rg_ratio, az_ratio = np.random.rand(), np.random.rand()
rg, az = int(rg_ratio*tomo_norm.shape[1]), int(az_ratio*tomo_norm.shape[0])
img_path = outpath + '/{}_tomosar_example_az_{}_rg_{}_{}.png'.format(pol,az, rg, tomo_method) 
tomo_plot(rg, az, slc_stack, tomo_norm, height, pol, img_path)#normalised

## Step 15: load lidar forest height (25 m resolution) and above-ground biomass (50 m resolution) data at radar geometry

In [ ]:
if terrain_cor_flag == 1 :
    lvis_rh_path = os.path.join(inpath, 'lidar_rh100_25m.npy')
    lvis_agb_path = os.path.join(inpath, 'lidar_agb_50m.npy')
    
    lvis_rh = np.load(lvis_rh_path)
    lvis_agb = np.load(lvis_agb_path)
    
    from skimage.transform import resize
    lvis_rh = resize(lvis_rh, (tomo_norm.shape[0], tomo_norm.shape[1]))
    lvis_agb = resize(lvis_agb, (tomo_norm.shape[0], tomo_norm.shape[1]))

## Step 16: overlap lidar height with tomosar slice

In [ ]:
if terrain_cor_flag == 1 :
    rg_ratio, az_ratio = np.random.rand(), np.random.rand()
    rg, az = int(rg_ratio*tomo_norm.shape[1]), int(az_ratio*tomo_norm.shape[0])
    
    img_path = outpath + '/{}_tomosar_example_az_{}_rg_{}_lidar.png'.format(pol,az, rg)    
    tomo_plot(rg, az, slc_stack, tomo_norm, height, pol, img_path, lvis_rh)#normalised

## Step 17: aggregate tomosar reflectivity profiles by lidar forest height and AGB

In [ ]:
if terrain_cor_flag == 1 :
    img_path = outpath + '/{}_tomosar_aggregated_profiles.png'.format(pol)    
    grouped_tomosar_profiles (tomo, lvis_rh, lvis_agb, z_vector, img_path)

## Step 18: compare tomosar phase centre with lidar height

In [ ]:
if terrain_cor_flag == 1 :
    img_path = outpath + '/lidar_height_{}_tomosar_phase_centre.png'.format(pol)    
    tomosar_phase_centre(tomo, lvis_rh, z_vector, img_path)

## Step 19: compare tomosar reflectivity at different height layers with lidar AGB

In [ ]:
if terrain_cor_flag == 1 :
    min_agb = 50 #Mg/ha
    img_path = outpath + '/lidar_agb_{}_tomosar_reflectivity_at'.format(pol)    
    tomosar_layerd_reflectivity (tomo, lvis_agb, min_agb, height, img_path)

## Your tasks: to compare results using
### (1) different polarization.
### (2) different tomography method.
### (3) different multilook size.
### (4) full dataset and a subset of the dataset.